In [15]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [16]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [17]:
# Global Parameters
R = 0.082057 # In atm*L/mol/K
N = 1.0 # In mols
gamma = 5.0/3.0 # Adiabatic index (5/3 for monoatomic gases)
pts = 100 # Number of points for each process

# Limits of parameters (volumes in L, pressures in atm)
v_min = 0.01
v_max = 20.0
p_min = 0.01
p_max = 20.0

In [18]:
def get_intersect(vi,pi,vf,pf,process1,process2):
    '''
    This function calculates the intersection point in the p-v diagram
    of a two step-process where each process is either isochoric,
    isobaric, isothermal or adiabatic. The intersection point is the
    point where the process changes. This calculation is done by considering
    each of the 12 possible combinations for which the point coordinates
    have been previously calculated analitically.
    
    Args:
    vi: float value for initial volume (in L)
    pi: float value for initial pressure (in atm)
    vf: float value for final volume (in L)
    pf: float value for final pressure (in atm)
    process1 = string to charactrize first process
    process2 = string to characterize second process
    
    Processes strings are: v -> isochoric, p -> isobaric, T -> isothermal, S -> adiabatic
    
    Returns:
    vc: float value for intersection point volume (in L)
    pc: float value for intersection point pressure (in atm)
    '''
    
    if process1 == 'v' and process2 == 'p':
        vc = vi
        pc = pf
    if process1 == 'p' and process2 == 'v':
        vc = vf
        pc = pi
    if process1 == 'v' and process2 == 'T':
        vc = vi
        pc = pf*vf/vc 
    if process1 == 'T' and process2 == 'v':
        vc = vf
        pc = pi*vi/vc
    if process1 == 'p' and process2 == 'T':
        pc = pi
        vc = vf*pf/pc
    if process1 == 'T' and process2 == 'p':
        pc = pf
        vc = vi*pi/pc
    if process1 == 'v' and process2 == 'S':
        vc = vi
        pc = pf*(vf/vc)**gamma
    if process1 == 'S' and process2 == 'v':
        vc = vf
        pc = pi*(vi/vc)**gamma
    if process1 == 'p' and process2 == 'S':
        pc = pi
        vc = (vf**gamma * pf/pc)**(1.0/gamma)
    if process1 == 'S' and process2 == 'p':
        pc = pf
        vc = (vi**gamma * pi/pc)**(1.0/gamma)
    if process1 == 'T' and process2 == 'S':
        vc = (pi*vi/pf/vf**gamma)**(1.0/(1.0-gamma))
        pc = pi*vi/vc
    if process1 == 'S' and process2 == 'T':
        vc = (pf*vf/pi/vi**gamma)**(1.0/(1.0-gamma))
    return vc,pc

In [19]:
def get_isotherm(v,pa,pb):
    '''
    This function calculates the pressure values for a isothermal
    processmstarting from pa to pb
    
    Inputs:
    v: 1d array of float volume values for which we are calculating p (in L)
    pa: float value for inital pressure (in atm)
    pb: float value for final pressure (in atm)
    
    Returns:
    p: 1d array of float pressure values (in atm)
    '''
    
    T = pa*v[0]/N/R
    p = N*R*T/v
    return p

In [20]:
def get_adiabatic(v,pa,pb):
    '''
    This function calculates the pressure values for a adiabatic
    process starting from pa to pb
    
    Inputs:
    v: 1d array of float volume values for which we are calculating p (in L)
    pa: float value for inital pressure (in atm)
    pb: float value for final pressure (in atm)
    
    Returns:
    p: 1d array of float pressure values (in atm)
    '''
    C = pa*v[0]**(gamma)
    p = C/v**(gamma)
    return p

In [21]:
def get_p_values(v,pa,pb,process):
    '''
    This function calculates the pressure values for a one step-process
    (either isochoric, isobaric, isothermal or adiabatic) starting from
    pa to pb.
    
    Inputs:
    v: 1d array of float v values for which we are calculating p (in L)
    pa: float value for inital pressure (in atm)
    pb: float value for final pressure (in atm)
    process: string to characterize process
    
    Returns:
    p: 1d array of float pressure values (in atm)
    '''
    if process == 'T':
        p = get_isotherm(v,pa,pb)
    if process == 'p':
        p = np.linspace(pa,pb,pts)
    if process == 'v':
        p = np.linspace(pa,pb,pts)
    if process == 'S':
        p = get_adiabatic(v,pa,pb)
    return p

In [22]:
def get_process_values(pstr):
    '''
    This functions translates from the long string that defines
    the two processes (which will be displayed on the interface)
    to two len=1 string values, each to characterize one process.
    The processes are characterized as:
    
    v: isochoric process
    p: isobaric process
    T: isothermal process
    S: adiabatic process
    
    Inputs:
    pstr: String that describes the two processes (ex. 'isochoric-isobaric')
    
    Returns:
    process1: string that characterizes the first process
    process2: string that characterizes the second process
    '''
    
    if pstr == 'isokoro-isobaro':
        process1, process2 = 'v','p'
    if pstr == 'isobaro-isokoro':
        process1, process2 = 'p','v'
    if pstr == 'isokoro-isotermo':
        process1, process2 = 'v','T'
    if pstr == 'isotermo-isokoro':
        process1, process2 = 'T','v'
    if pstr == 'isobaro-isotermo':
        process1, process2 = 'p','T'
    if pstr == 'isotermo-isobaro':
        process1, process2 = 'T','p'
    if pstr == 'isokoro-adiabatiko':
        process1, process2 = 'v', 'S'
    if pstr == 'adiabatiko-isokoro':
        process1, process2 = 'S', 'v'
    if pstr == 'isobaro-adiabatiko':
        process1, process2 = 'p', 'S'
    if pstr == 'adiabatiko-isobaro':
        process1, process2 = 'S', 'p'
    if pstr == 'isotermo-adiabatiko':
        process1, process2 = 'T', 'S'
    if pstr == 'adiabatiko-isotermo':
        process == 'S', 'T'
    return process1, process2

In [23]:
def calculate_work(v1,p1,v2,p2):
    '''
    This function calculates the total Work done in the process
    described by the point arrays (v1,p1) and (v2,p2). This calculation
    is done by applying numerical integration based on:
    
    dW = -pdV
    
    The fact that all values in v1 or v2 are equidistant is exploited to
    simplify the calculation.
    
    Inputs:
    v1: 1d array of float volume values for first process (in L)
    p1: 1d array of float pressure values for first process (in atm)
    v2: 1d array of float volume values for second process (in L)
    p2: 1d array of float pressure values for second process (in atm)
    
    Returns:
    W: float value for the work done by the system (negative) or on the system (positive) (in J)
    
    '''
    W= -sum(p1)*(v1[pts-1] - v1[0])/pts - sum(p2)*(v2[pts-1] - v2[0])/pts
    return W

In [24]:
def calculate_energy(vi,pi,vf,pf):
    '''
    This function calculates the total internal-energy change in any process
    that goes from (vi,pi) to (vf,pf). This calculation
    is done exploiting the fact that internal-energy is a function of state.
    
    dU = 3/2*N*R*dT
    
    Inputs:
    vi: float value for initial volume (in L)
    pi: float value for initial pressure (in atm)
    vf: float value for final volume (in L)
    pf: float value for final pressure (in atm)  
    
    Returns:
    dU: float value for the internal-energy change of the system (in J)
    '''
    dU = 1.5*(vf*pf - vi*pi)
    return dU

In [25]:
def update_figure(change):
    '''
    This functions recalculates the marks and output values
    whenever a widget values is changed.
    '''
    
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    pf = pf_slider.value
    process_string = process_sel.value
    
    
    process1,process2 = get_process_values(process_string)
    

    vc,pc = get_intersect(vi,pi,vf,pf,process1,process2)
    v1 = np.linspace(vi,vc,pts)
    v2 = np.linspace(vc,vf,pts)
    p1 = get_p_values(v1,pi,pc,process1)
    p2 = get_p_values(v2,pc,pf,process2)
    
    Ti = vi*pi/N/R
    Ti_text = '%.2f' % Ti
    Tf = vf*pf/N/R
    Tf_text = '%.2f' % Tf
    Ti_widget.value = Ti_text
    Tf_widget.value = Tf_text
    
    work = calculate_work(v1,p1,v2,p2)
    energy = calculate_energy(vi,pi,vf,pf)
    
    work_value_text.value = '%.2f' % work
    energy_value_text.value = '%.2f' % energy
    
    heat_value_text.value = '%.2f' % (energy - work)
        
    x_values = []
    y_values = []
    for i in range(pts):
        x_values.append(v1[i])
        y_values.append(p1[i])
    for i in range(pts):
        x_values.append(v2[i])
        y_values.append(p2[i])
        
    curve.x = x_values
    curve.y = y_values
    InitialPoint.x = [vi]
    InitialPoint.y = [pi]
    FinalPoint.x = [vf]
    FinalPoint.y = [pf]

In [26]:
# Default initial state
vi = 1.0
pi = 1.0
Ti = vi*pi/N/R
Ti_text = '%.2f' % Ti
process1 = 'v'
process2 = 'p'

# Default final state
vf = 2.0
pf = 0.8
Tf = vf*pf/N/R
Tf_text = '%.2f' % Tf

# Get curves
vc,pc = get_intersect(vi,pi,vf,pf,process1,process2)
v1 = np.linspace(vi,vc,pts)
v2 = np.linspace(vc,vf,pts)
p1 = get_p_values(v1,pi,pc,process1)
p2 = get_p_values(v2,pc,pf,process2)


In [27]:
########################
###CREATE THE FIGURES###
########################

fig_122_001 = bq.Figure(title='Prozesu konposatua p-v diagraman',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
    )


scale_x = bqs.LinearScale(min = v_min, max = v_max)
scale_y = bqs.LinearScale(min = p_min, max = p_max)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_122_001.axes = [axis_x,axis_y]


########################
####CREATE THE MARKS####
########################

marks = []

x_values = []
y_values = []
for i in range(pts):
    x_values.append(v1[i])
    y_values.append(p1[i])
for i in range(pts):
    x_values.append(v2[i])
    y_values.append(p2[i])

color = ['#0079c4']

curve = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y},
                colors = color,
                fill_colors = ['green'],
                fill_opacities = [0.35],
                fill = 'bottom',
                labels = ['Prozesua'],
                display_legend=True
)

marks.append(curve)

InitialPoint = bqm.Scatter(
    name = 'Initial Point',
    x = [vi],
    y = [pi],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['orange'],
    names = [],
    labels=['Hasierako egoera'],
    display_legend = True
)      

marks.append(InitialPoint)

FinalPoint = bqm.Scatter(
    name = 'Final Point',
    x = [vf],
    y = [pf],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=['Bukaerako egoera'],
    display_legend = True
)      

marks.append(FinalPoint)

fig_122_001.marks = marks


########################
######  WIDGETS  #######
########################

vi_slider = widgets.FloatSlider(
    value=1.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

vf_slider = widgets.FloatSlider(
    value=2.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_f$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

pi_slider = widgets.FloatSlider(
    value=1.0,
    min=p_min,
    max=p_max,
    step=0.1,
    description='$p_i$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

pf_slider = widgets.FloatSlider(
    value=2.0,
    min=p_min,
    max=p_max,
    step=0.1,
    description='$p_f$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

process_sel = widgets.Dropdown(
    options=['isokoro-isobaro', 'isobaro-isokoro', 'isokoro-isotermo', 'isotermo-isokoro', 'isobaro-isotermo', 'isotermo-isobaro',
             'isokoro-adiabatiko','adiabatiko-isokoro','isobaro-adiabatiko','adiabatiko-isobaro','isotermo-adiabatiko','adiabatiko-isotermo'],
    value='isokoro-isobaro',
    description='Prozesuak',
    disabled=False,
)

vi_slider.observe(update_figure, 'value')
pi_slider.observe(update_figure, 'value')
vf_slider.observe(update_figure, 'value')
pf_slider.observe(update_figure, 'value')
process_sel.observe(update_figure, 'value')

Ti_widget = widgets.HTML(
    value=Ti_text
)

Tf_widget = widgets.HTML(
    value=Tf_text
)




########################
######  OUTPUTS  #######
########################

work_value_text = widgets.HTML(
    value="",
)

energy_value_text = widgets.HTML(
    value="",
) 

heat_value_text = widgets.HTML(
    value="",
) 




########################
######  LAYOUT  ########
########################

## Left Block ##
left_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_122_000.children = [widgets.Label(value="Hasierako egoera"), vi_slider, pi_slider, widgets.HBox([widgets.Label(value='$T_i:$'),Ti_widget,widgets.Label(value='$K$')]),
                               widgets.Label(value="Bukaerako egoera"), vf_slider, pf_slider, widgets.HBox([widgets.Label(value='$T_f:$'),Tf_widget,widgets.Label(value='$K$')]),
                               process_sel]

## Figure Block ##
figure_block_122_000 = widgets.VBox([fig_122_001], layout=widgets.Layout(width='60%'))
figure_block_122_000.children = [fig_122_001]

## Right Block ##
right_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%'))
right_block_122_000.children = [
                       widgets.HBox([widgets.Label(value="$W:$"),work_value_text,widgets.Label(value="$J$")]),
                       widgets.HBox([widgets.Label(value="$\Delta U:$"),energy_value_text,widgets.Label(value="$J$")]),
                       widgets.HBox([widgets.Label(value="$Q:$"),heat_value_text,widgets.Label(value="$J$")])
                      ]


## Main Block ##

main_block_122_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_122_000.children = [left_block_122_000, figure_block_122_000,right_block_122_000]

main_block_122_000

In [28]:
'''
Hobetzeko:
Azalerak ondo begiratu, batzutan ez du zentzurik grafikatzen denak
'''

'\nHobetzeko:\nAzalerak ondo begiratu, batzutan ez du zentzurik grafikatzen denak\n'